In [214]:
import pandas as pd 
import requests
from bs4 import BeautifulSoup

In [215]:
URL = "https://www.zumper.com/apartments-for-rent/vancouver-bc/west-end/studios"
requestURL = requests.get(URL)

In [216]:
content = requestURL.content

In [217]:
soup = BeautifulSoup(content, "html.parser")

In [218]:
rent = [x.text for x in soup.find_all("div", class_="css-uzc059")]
bath = [x.text for x in soup.find_all("div", class_="css-1iq6kk8")]



In [219]:
dict = {'Monthly': rent, 'Number of baths': type}
df= pd.DataFrame(dict)

In [220]:
df['Monthly'] = df['Monthly'].str.replace('\D+', '')

C:\Users\Jimmy\AppData\Local\Temp/ipykernel_12496/424978522.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Monthly'] = df['Monthly'].str.replace('\D+', '')


In [221]:
df

,Monthly,Number of baths
0,1900,[Studio • 1 Bath]
1,2369,[Studio • 1 Bath]
2,2268,[Studio • 1 Bath]
3,2570,[Studio • 1 Bath]
4,2650,[Studio • 1 Bath]
5,2450,[Studio • 1 Bath]
6,2495,[Studio • 1 Bath]
7,3500,[Studio • 1 Bath]
8,2700,[Studio • 1 Bath]
9,2375,[Studio • 1 Bath]


In [267]:
URL2 = "https://www.kijiji.ca/b-apartments-condos/vancouver-downtown-west-end/studios/k0c37l1700292?rb=true"
requestURL2 = requests.get(URL2)
content2 = requestURL2.content

soup2 = BeautifulSoup(content2, "html.parser")

In [241]:
rent2 = [x.text for x in soup2.findall("div", class_= "price")]

TypeError: 'NoneType' object is not callable

In [288]:
main_container = soup2.find("div",id="MainContainer")
main_containerContent = main_container.find("div", id="mainPageContent")
realEstate = main_containerContent.find("div", class_="layout-3 new-real-estate-srp")




In [289]:
realEstate

<div class="layout-3 new-real-estate-srp">
<div class="fes-pagelet">
<div data-fes-id="ResultsHeader"><div class="resultsHeaderContainer-3418577612"><div class="resultsHeadingContainer-781502691"><h1 class="resultsHeading-1285903303">Listings for "studios" in Apartments, Condos &amp; Houses For Rent in Downtown-West End</h1><span class="resultsShowingCount-1707762110">Showing 1 - 20 of 20 results</span></div><div class="filterContainer-246224382"><button class="addFiltersButton-2536109863 button-1997310527 button__medium-1066667140" type="button"><svg class="icon-459822882 icon-2323234912" height="100%" width="100%"><use xlink:href="#icon-plus"></use></svg><span class="addFiltersText-2852895903">Add Filters</span></button><span class="loadingChip-635405894"></span><span class="loadingChip-635405894"></span><span class="loadingChip-635405894"></span></div><div class="resultsHeaderControls-829581855"><form class="container-521757047" enctype="application/x-www-form-urlencoded" id="alertS